In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import os
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
import requests
import PyPDF2
import re

In [2]:
websites = ['https://cbv.com.br/superliga/equipes/equipes-masculino/', 'https://cbv.com.br/superliga/equipes/equipes-feminino/', 'https://cbv.com.br/superligab/equipes/']
links = []
arquivos = []
PDFs = []

In [3]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

for website in websites:
    driver.get(website)
    driver.maximize_window()

    elementos_div = driver.find_elements(By.XPATH, '//div[@class="row table-item a"]')

    for elemento_div in elementos_div:
        elemento_a = elemento_div.find_element(By.TAG_NAME, 'a')
        link = elemento_a.get_attribute('href')
        links.append(link)
    
    elementos_div2 = driver.find_elements(By.XPATH, '//div[@class="row table-item b"]')

    for elemento_div2 in elementos_div2:
        elemento_a2 = elemento_div2.find_element(By.TAG_NAME, 'a')
        link = elemento_a2.get_attribute('href')
        links.append(link)

In [4]:
links.clear()

In [5]:
print(links)
len(links)

[]


0

In [6]:
diretorio_de_destino = 'C:\\Users\\GabrielBeranRodrigue\\Desktop\\Exato\\Pyhon\\Download Arquivos'  # Substitua pelo caminho desejado

for url in links:
    # Obtenha o nome do arquivo a partir da URL
    nome_do_arquivo = url.split("/")[-1]

    # Adicione o nome do arquivo à lista de arquivos
    arquivos.append(nome_do_arquivo)

    # Crie o caminho completo do arquivo de destino
    caminho_completo = os.path.join(diretorio_de_destino, nome_do_arquivo)

    # Faça o download do PDF
    response = requests.get(url)

    # Verifique se a solicitação foi bem-sucedida (código de status 200)
    if response.status_code == 200:
        # Abra o arquivo no modo binário e escreva o conteúdo do PDF nele
        with open(caminho_completo, 'wb') as arquivo:
            arquivo.write(response.content)
        print(f"Download de {nome_do_arquivo} concluído.")
    else:
        print(f"Erro ao baixar {nome_do_arquivo} - Status de resposta: {response.status_code}")

print("Todos os downloads concluídos.")

# Agora a lista de arquivos contém os nomes de todos os arquivos baixados
print("Nomes dos arquivos baixados:", arquivos)

Todos os downloads concluídos.
Nomes dos arquivos baixados: []


In [7]:
print(arquivos)

[]


In [8]:
for arquivo in arquivos:
    caminho_arquivo_pdf = f'C:\\Users\\GabrielBeranRodrigue\\Desktop\\Exato\\Pyhon\\Download Arquivos\\{arquivo}' # Altere para o caminho onde os arquivos se encontram, mas mantenha o final \\{arquivo}

    # Abra o arquivo PDF em modo de leitura binária
    with open(caminho_arquivo_pdf, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Verifique o número de páginas no PDF
        num_paginas = len(pdf_reader.pages)

        texto_completo = ''  # Inicialize uma string para armazenar o texto completo do PDF

        # Extraia o texto de cada página
        for pagina_numero in range(num_paginas):
            pagina = pdf_reader.pages[pagina_numero]
            texto = pagina.extract_text()
            texto_completo += texto  # Adicione o texto da página ao texto completo

        PDFs.append(texto_completo)  # Adicione o texto completo à lista de PDFs

# Agora a lista PDFs contém o texto completo de cada arquivo PDF
for i, pdf_text in enumerate(PDFs):
    print(f"Texto do arquivo {arquivos[i]}:\n{pdf_text}\n")

In [9]:
len(PDFs)

0

In [10]:
def remover_cabecalho(texto):
    padrao = r"NºNome do jogador"
    correspondencia = re.search(padrao, texto)
    if correspondencia:
        texto_limpo = texto[correspondencia.end():]
    else:
        texto_limpo = texto
    return texto_limpo

# Remova o cabeçalho de cada item na lista PDFs
PDFs_limpos = [remover_cabecalho(texto) for texto in PDFs]

# Imprima a lista resultante
for texto_limpo in PDFs_limpos:
    print(texto_limpo)

In [11]:
PDFs_limpos_filtrados = []
PDFs_sobra = []

for pdf in PDFs_limpos:
    # Divida o texto em linhas
    linhas = pdf.split('\n')
    
    # Encontre o índice da linha que contém "LÍBERO"
    indices_libero = [i for i, linha in enumerate(linhas) if "LíBERO" in linha]

    if indices_libero:
        # Encontre o índice mais baixo (primeira ocorrência de "LÍBERO")
        indice_libero = min(indices_libero)
        
        # Mantenha apenas as linhas até "LÍBERO"
        pdf_limpo = '\n'.join(linhas[:indice_libero])
        PDFs_limpos_filtrados.append(pdf_limpo)
        
        # Salve o conteúdo restante após "LÍBERO"
        pdf_sobra = '\n'.join(linhas[indice_libero:])
        PDFs_sobra.append(pdf_sobra)
    else:
        # Se "LÍBERO" não foi encontrado, mantenha o texto original em ambas as listas
        PDFs_limpos_filtrados.append(pdf)
        PDFs_sobra.append(pdf)

# Imprima a lista resultante
#for pdf_limpo in PDFs_limpos_filtrados:
    #print(pdf_limpo)
    
print("### Conteúdo restante após LÍBERO ###")
for pdf_sobra in PDFs_sobra:
    print(pdf_sobra)

### Conteúdo restante após LÍBERO ###


In [12]:
# Inicialize uma lista vazia para armazenar os nomes limpos
nomes_limpos = []

# Use expressões regulares para remover números e a frase "NºNome do jogador"
for item in PDFs_limpos_filtrados:
    nome_limpo = re.sub(r'\d+|NºNome do jogador', '', item).strip()
    if nome_limpo:  # Verifique se a string resultante não está vazia
        nomes_limpos.append(nome_limpo)

# Imprima a lista de nomes limpos
print(nomes_limpos)

[]


In [13]:
# Substitua os caracteres de quebra de linha por vírgulas em cada valor
nomes_com_virgulas = [nome.replace('\n', ', ') for nome in nomes_limpos]

# Imprima a lista com vírgulas
print(nomes_com_virgulas)
len(nomes_com_virgulas)

[]


0

In [14]:
nomes_divididos = []

# Itere sobre a lista e divida cada valor por ", "
for valor in nomes_com_virgulas:
    nomes = valor.split(', ')
    nomes_divididos.extend(nomes)  # Adicione os nomes à lista final

# Imprima a lista de nomes divididos
nomes_divididos = list(set(nomes_divididos))
print(nomes_divididos)
len(nomes_divididos)

[]


0

In [15]:
# Inicialize as listas de resultado
nomes_banco = []
PDFs_sobra2 = []

for pdf in PDFs_sobra:
    # Divida o texto em linhas
    linhas = pdf.split('\n')
    
    # Encontre o índice da linha que contém "LÍBERO"
    indices_libero = [i for i, linha in enumerate(linhas) if "ASSINATURA" in linha]

    if indices_libero:
        # Encontre o índice mais baixo (primeira ocorrência de "LÍBERO")
        indice_libero = min(indices_libero)
        
        # Mantenha apenas as linhas até "LÍBERO"
        pdf_limpo = '\n'.join(linhas[:indice_libero])
        nomes_banco.append(pdf_limpo)
        
        # Salve o conteúdo restante após "LÍBERO"
        pdf_sobra = '\n'.join(linhas[indice_libero:])
        PDFs_sobra2.append(pdf_sobra)
    else:
        # Se "LÍBERO" não foi encontrado, mantenha o texto original em ambas as listas
        nomes_banco.append(pdf)
        PDFs_sobra2.append(pdf)

 #Imprima a lista resultante
#for pdf_limpo in nomes_banco:
    #print(pdf_limpo)
    
# Usar lista PDFs_sobra2 no futuro se necessário
print("### Conteúdo restante após ASSINATURA ###")
for pdf_sobra in PDFs_sobra2:
    print(pdf_sobra)

### Conteúdo restante após ASSINATURA ###


In [16]:
print(nomes_banco)

[]


In [17]:
nomes_banco_fitrados = []
# Padrão de expressão regular para encontrar números
padrao_numeros = r'\d+'

for valor in nomes_banco:
    # Use a expressão regular para remover números
    valor_sem_numeros = re.sub(padrao_numeros, '', valor)

    # Remova a palavra "LíBERO" e a frase "MEMBROS OFICIAIS ADMITIDOS NO BANCO"
    valor_filtrado = valor_sem_numeros.replace("LíBERO", "").replace("MEMBROS OFICIAIS ADMITIDOS NO BANCO", "")

    # Adicione o valor filtrado à lista de resultados
    nomes_banco_fitrados.append(valor_filtrado.strip())

# Imprima a lista resultante
for nome in nomes_banco_fitrados:
    print(nome)

In [18]:
nomes_banco_filtrados2 = []

prefixos = ["L", "AT", "F", "PF", "T"]

for valor in nomes_banco_fitrados:
    # Divida o valor em linhas
    linhas = valor.split('\n')

    # Inicialize uma lista para as linhas filtradas
    linhas_filtradas = []

    for linha in linhas:
        # Remova os prefixos
        for prefixo in prefixos:
            if linha.startswith(prefixo):
                linha = linha[len(prefixo):]

        # Adicione a linha filtrada à lista de linhas
        linhas_filtradas.append(linha.strip())

    # Combine as linhas filtradas em um único valor
    valor_filtrado = '\n'.join(linhas_filtradas)

    # Adicione o valor filtrado à lista de resultados
    nomes_banco_filtrados2.append(valor_filtrado)

# Imprima a lista resultante
for nome in nomes_banco_filtrados2:
    print(nome)

In [19]:
# Inicializa a lista de nomes finais
nomes_banco_final = []

# Divide as linhas e adiciona à lista de nomes finais
for linha in nomes_banco_filtrados2:
    nomes = linha.split('\n')
    nomes_formatados = ["'" + nome + "'" for nome in nomes if nome.strip()]  # Remove linhas vazias
    if nomes_formatados:
        nomes_banco_final.append(", ".join(nomes_formatados))

print(nomes_banco_final)
len(nomes_banco_final)

[]


0

In [20]:
# Lista vazia para armazenar os novos valores
nova_lista = []

# Iterar pela lista e transformar os valores entre aspas simples em novos valores
for item in nomes_banco_final:
    valores = item.split("', '")  # Divide a string com base em "', '"
    for valor in valores:
        novo_valor = valor.strip("'")  # Remove as aspas simples do início e do fim
        nova_lista.append(novo_valor)

# Imprima a nova lista
print(nova_lista)
len(nova_lista)

[]


0

In [21]:
# Unir as listas
lista_unida = nomes_divididos + nova_lista
lista_unida = list(set(lista_unida))

print(lista_unida)
len(lista_unida)

[]


0

In [22]:
# Código para importar arquivos de saida e analisá-los
# Caminho do arquivo de saída
#caminho_arquivo_saida = 'C:\\Users\\GabrielBeranRodrigue\\Desktop\\Exato\\Pyhon\\Download Arquivos\\PDFs_sobra2.txt'

# Conteúdo a ser salvo no arquivo
#conteudo_para_salvar = '\n\n'.join(PDFs_sobra2)

# Abra o arquivo no modo de escrita e salve o conteúdo
#with open(caminho_arquivo_saida, 'w', encoding='utf-8') as arquivo_saida:
#    arquivo_saida.write(conteudo_para_salvar)

#print(f"Conteúdo salvo em {caminho_arquivo_saida}")

In [23]:
df_list = pd.DataFrame({'Membro': lista_unida})
df_list.to_csv('Membros CBV Vôlei Teste.csv', index=False)